In [1]:
%cd capstone-project-9900w12achatllm

/gemini/code/capstone-project-9900w12achatllm


In [2]:
!pip install -e ".[torch,metrics]"
!pip install --upgrade pandas pyarrow datasets
!pip install auto_gptq>=0.5.0
!pip install optimum
!pip uninstall apex -y

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Obtaining file:///gemini/code/capstone-project-9900w12achatllm
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 20.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 88.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.1/314.1 kB 58.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 40.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.8/245.8 kB 2.1 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 2.2 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 10.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 802.6 kB/s

batch size = 3, max_example = 250

In [3]:
!llamafactory-cli train \
    --stage sft \
    --do_train True \
    --model_name_or_path model/Qwen2-1.5B-Instruct \
    --preprocessing_num_workers 16 \
    --finetuning_type lora \
    --template qwen \
    --flash_attn auto \
    --dataset_dir data \
    --dataset MedQA_train,PubMedQA_pqal_train \
    --cutoff_len 1024 \
    --learning_rate 5e-05 \
    --num_train_epochs 3.0 \
    --max_samples 250 \
    --per_device_train_batch_size 3 \
    --gradient_accumulation_steps 8 \
    --lr_scheduler_type cosine \
    --max_grad_norm 1.0 \
    --logging_steps 5 \
    --save_steps 1000 \
    --warmup_steps 0 \
    --optim adamw_torch \
    --packing False \
    --report_to none \
    --output_dir saves/Qwen2-1.5B-Instruct/lora/train_2024-07-08-20-41-25 \
    --fp16 True \
    --plot_loss True \
    --ddp_timeout 180000000 \
    --include_num_input_tokens_seen True \
    --lora_rank 8 \
    --lora_alpha 16 \
    --lora_dropout 0 \
    --lora_target all \
    --val_size 0.1 \
    --eval_strategy steps \
    --eval_steps 5 \
    --per_device_eval_batch_size 2

[2024-07-11 05:19:47,083] [INFO] [real_accelerator.py:158:get_accelerator] Setting ds_accelerator to cuda (auto detect)
Using config file: /etc/orion/env/env.conf
07/11/2024 05:19:51 - INFO - llamafactory.hparams.parser - Process rank: 0, device: cuda:0, n_gpu: 1, distributed training: False, compute dtype: torch.float16
[INFO|tokenization_utils_base.py:2159] 2024-07-11 05:19:51,905 >> loading file vocab.json
[INFO|tokenization_utils_base.py:2159] 2024-07-11 05:19:51,905 >> loading file merges.txt
[INFO|tokenization_utils_base.py:2159] 2024-07-11 05:19:51,905 >> loading file tokenizer.json
[INFO|tokenization_utils_base.py:2159] 2024-07-11 05:19:51,905 >> loading file added_tokens.json
[INFO|tokenization_utils_base.py:2159] 2024-07-11 05:19:51,905 >> loading file special_tokens_map.json
[INFO|tokenization_utils_base.py:2159] 2024-07-11 05:19:51,905 >> loading file tokenizer_config.json
[WARNING|logging.py:313] 2024-07-11 05:19:52,195 >> Special tokens have been added in the vocabulary, 

In [4]:
!llamafactory-cli train \
    --stage sft \
    --model_name_or_path model/Qwen2-1.5B-Instruct \
    --preprocessing_num_workers 16 \
    --finetuning_type lora \
    --template qwen \
    --flash_attn auto \
    --dataset_dir data \
    --dataset MedQA_test,PubMedQA_pqal_test \
    --cutoff_len 1024 \
    --max_samples 100 \
    --per_device_eval_batch_size 2 \
    --predict_with_generate True \
    --max_new_tokens 512 \
    --top_p 0.7 \
    --temperature 0.95 \
    --output_dir saves/Qwen2-1.5B-Instruct/lora/eval_2024-07-09-00-26-34 \
    --do_predict True \
    --adapter_name_or_path saves/Qwen2-1.5B-Instruct/lora/train_2024-07-08-20-41-25

[2024-07-11 05:32:04,480] [INFO] [real_accelerator.py:158:get_accelerator] Setting ds_accelerator to cuda (auto detect)
Using config file: /etc/orion/env/env.conf
07/11/2024 05:32:08 - INFO - llamafactory.hparams.parser - Process rank: 0, device: cuda:0, n_gpu: 1, distributed training: False, compute dtype: None
[INFO|tokenization_utils_base.py:2159] 2024-07-11 05:32:08,686 >> loading file vocab.json
[INFO|tokenization_utils_base.py:2159] 2024-07-11 05:32:08,686 >> loading file merges.txt
[INFO|tokenization_utils_base.py:2159] 2024-07-11 05:32:08,686 >> loading file tokenizer.json
[INFO|tokenization_utils_base.py:2159] 2024-07-11 05:32:08,686 >> loading file added_tokens.json
[INFO|tokenization_utils_base.py:2159] 2024-07-11 05:32:08,686 >> loading file special_tokens_map.json
[INFO|tokenization_utils_base.py:2159] 2024-07-11 05:32:08,686 >> loading file tokenizer_config.json
[WARNING|logging.py:313] 2024-07-11 05:32:08,969 >> Special tokens have been added in the vocabulary, make sure

batch size = 3, max_exmaple = 500

In [5]:
!llamafactory-cli train \
    --stage sft \
    --do_train True \
    --model_name_or_path model/Qwen2-1.5B-Instruct \
    --preprocessing_num_workers 16 \
    --finetuning_type lora \
    --template qwen \
    --flash_attn auto \
    --dataset_dir data \
    --dataset MedQA_train,PubMedQA_pqal_train \
    --cutoff_len 1024 \
    --learning_rate 5e-05 \
    --num_train_epochs 3.0 \
    --max_samples 500 \
    --per_device_train_batch_size 3 \
    --gradient_accumulation_steps 8 \
    --lr_scheduler_type cosine \
    --max_grad_norm 1.0 \
    --logging_steps 5 \
    --save_steps 1000 \
    --warmup_steps 0 \
    --optim adamw_torch \
    --packing False \
    --report_to none \
    --output_dir saves/Qwen2-1.5B-Instruct/lora/train_2024-07-08-20-41-26 \
    --fp16 True \
    --plot_loss True \
    --ddp_timeout 180000000 \
    --include_num_input_tokens_seen True \
    --lora_rank 8 \
    --lora_alpha 16 \
    --lora_dropout 0 \
    --lora_target all \
    --val_size 0.1 \
    --eval_strategy steps \
    --eval_steps 5 \
    --per_device_eval_batch_size 2

[2024-07-11 05:37:06,742] [INFO] [real_accelerator.py:158:get_accelerator] Setting ds_accelerator to cuda (auto detect)
Using config file: /etc/orion/env/env.conf
07/11/2024 05:37:11 - INFO - llamafactory.hparams.parser - Process rank: 0, device: cuda:0, n_gpu: 1, distributed training: False, compute dtype: torch.float16
[INFO|tokenization_utils_base.py:2159] 2024-07-11 05:37:11,262 >> loading file vocab.json
[INFO|tokenization_utils_base.py:2159] 2024-07-11 05:37:11,262 >> loading file merges.txt
[INFO|tokenization_utils_base.py:2159] 2024-07-11 05:37:11,262 >> loading file tokenizer.json
[INFO|tokenization_utils_base.py:2159] 2024-07-11 05:37:11,263 >> loading file added_tokens.json
[INFO|tokenization_utils_base.py:2159] 2024-07-11 05:37:11,263 >> loading file special_tokens_map.json
[INFO|tokenization_utils_base.py:2159] 2024-07-11 05:37:11,263 >> loading file tokenizer_config.json
[WARNING|logging.py:313] 2024-07-11 05:37:11,563 >> Special tokens have been added in the vocabulary, 

In [6]:
!llamafactory-cli train \
    --stage sft \
    --model_name_or_path model/Qwen2-1.5B-Instruct \
    --preprocessing_num_workers 16 \
    --finetuning_type lora \
    --template qwen \
    --flash_attn auto \
    --dataset_dir data \
    --dataset MedQA_test,PubMedQA_pqal_test \
    --cutoff_len 1024 \
    --max_samples 100 \
    --per_device_eval_batch_size 2 \
    --predict_with_generate True \
    --max_new_tokens 512 \
    --top_p 0.7 \
    --temperature 0.95 \
    --output_dir saves/Qwen2-1.5B-Instruct/lora/eval_2024-07-09-00-26-35 \
    --do_predict True \
    --adapter_name_or_path saves/Qwen2-1.5B-Instruct/lora/train_2024-07-08-20-41-26

[2024-07-11 05:56:22,322] [INFO] [real_accelerator.py:158:get_accelerator] Setting ds_accelerator to cuda (auto detect)
Using config file: /etc/orion/env/env.conf
07/11/2024 05:56:26 - INFO - llamafactory.hparams.parser - Process rank: 0, device: cuda:0, n_gpu: 1, distributed training: False, compute dtype: None
[INFO|tokenization_utils_base.py:2159] 2024-07-11 05:56:26,489 >> loading file vocab.json
[INFO|tokenization_utils_base.py:2159] 2024-07-11 05:56:26,489 >> loading file merges.txt
[INFO|tokenization_utils_base.py:2159] 2024-07-11 05:56:26,489 >> loading file tokenizer.json
[INFO|tokenization_utils_base.py:2159] 2024-07-11 05:56:26,489 >> loading file added_tokens.json
[INFO|tokenization_utils_base.py:2159] 2024-07-11 05:56:26,489 >> loading file special_tokens_map.json
[INFO|tokenization_utils_base.py:2159] 2024-07-11 05:56:26,489 >> loading file tokenizer_config.json
[WARNING|logging.py:313] 2024-07-11 05:56:26,763 >> Special tokens have been added in the vocabulary, make sure

batch size = 3, max_example = 1000

In [7]:
!llamafactory-cli train \
    --stage sft \
    --do_train True \
    --model_name_or_path model/Qwen2-1.5B-Instruct \
    --preprocessing_num_workers 16 \
    --finetuning_type lora \
    --template qwen \
    --flash_attn auto \
    --dataset_dir data \
    --dataset MedQA_train,PubMedQA_pqal_train \
    --cutoff_len 1024 \
    --learning_rate 5e-05 \
    --num_train_epochs 3.0 \
    --max_samples 1000 \
    --per_device_train_batch_size 3 \
    --gradient_accumulation_steps 8 \
    --lr_scheduler_type cosine \
    --max_grad_norm 1.0 \
    --logging_steps 5 \
    --save_steps 1000 \
    --warmup_steps 0 \
    --optim adamw_torch \
    --packing False \
    --report_to none \
    --output_dir saves/Qwen2-1.5B-Instruct/lora/train_2024-07-08-20-41-27 \
    --fp16 True \
    --plot_loss True \
    --ddp_timeout 180000000 \
    --include_num_input_tokens_seen True \
    --lora_rank 8 \
    --lora_alpha 16 \
    --lora_dropout 0 \
    --lora_target all \
    --val_size 0.1 \
    --eval_strategy steps \
    --eval_steps 5 \
    --per_device_eval_batch_size 2

[2024-07-11 06:01:06,900] [INFO] [real_accelerator.py:158:get_accelerator] Setting ds_accelerator to cuda (auto detect)
Using config file: /etc/orion/env/env.conf
07/11/2024 06:01:10 - INFO - llamafactory.hparams.parser - Process rank: 0, device: cuda:0, n_gpu: 1, distributed training: False, compute dtype: torch.float16
[INFO|tokenization_utils_base.py:2159] 2024-07-11 06:01:10,846 >> loading file vocab.json
[INFO|tokenization_utils_base.py:2159] 2024-07-11 06:01:10,846 >> loading file merges.txt
[INFO|tokenization_utils_base.py:2159] 2024-07-11 06:01:10,846 >> loading file tokenizer.json
[INFO|tokenization_utils_base.py:2159] 2024-07-11 06:01:10,846 >> loading file added_tokens.json
[INFO|tokenization_utils_base.py:2159] 2024-07-11 06:01:10,846 >> loading file special_tokens_map.json
[INFO|tokenization_utils_base.py:2159] 2024-07-11 06:01:10,846 >> loading file tokenizer_config.json
[WARNING|logging.py:313] 2024-07-11 06:01:11,114 >> Special tokens have been added in the vocabulary, 

In [ ]:
!llamafactory-cli train \
    --stage sft \
    --model_name_or_path model/Qwen2-1.5B-Instruct \
    --preprocessing_num_workers 16 \
    --finetuning_type lora \
    --template qwen \
    --flash_attn auto \
    --dataset_dir data \
    --dataset MedQA_test,PubMedQA_pqal_test \
    --cutoff_len 1024 \
    --max_samples 100 \
    --per_device_eval_batch_size 2 \
    --predict_with_generate True \
    --max_new_tokens 512 \
    --top_p 0.7 \
    --temperature 0.95 \
    --output_dir saves/Qwen2-1.5B-Instruct/lora/eval_2024-07-09-00-26-36 \
    --do_predict True \
    --adapter_name_or_path saves/Qwen2-1.5B-Instruct/lora/train_2024-07-08-20-41-27

[2024-07-11 06:33:15,513] [INFO] [real_accelerator.py:158:get_accelerator] Setting ds_accelerator to cuda (auto detect)
Using config file: /etc/orion/env/env.conf
07/11/2024 06:33:19 - INFO - llamafactory.hparams.parser - Process rank: 0, device: cuda:0, n_gpu: 1, distributed training: False, compute dtype: None
[INFO|tokenization_utils_base.py:2159] 2024-07-11 06:33:19,735 >> loading file vocab.json
[INFO|tokenization_utils_base.py:2159] 2024-07-11 06:33:19,735 >> loading file merges.txt
[INFO|tokenization_utils_base.py:2159] 2024-07-11 06:33:19,735 >> loading file tokenizer.json
[INFO|tokenization_utils_base.py:2159] 2024-07-11 06:33:19,735 >> loading file added_tokens.json
[INFO|tokenization_utils_base.py:2159] 2024-07-11 06:33:19,735 >> loading file special_tokens_map.json
[INFO|tokenization_utils_base.py:2159] 2024-07-11 06:33:19,735 >> loading file tokenizer_config.json
[WARNING|logging.py:313] 2024-07-11 06:33:20,042 >> Special tokens have been added in the vocabulary, make sure